In [2]:
import json
import pandas as pd
from collections import defaultdict
import numpy as np
import re

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f,encoding="utf-8")
    
names_gender=load_simple_json('de/all_person_polyglot_new.json')#394
df=pd.DataFrame.from_dict(names_gender,orient="index")
print len(df),len(names_gender)
about_pages=pd.read_csv('de/all_data_bias_labour_gender.csv',sep='\t', encoding='utf-8',index_col=0)#885
df=pd.concat([about_pages,df],join='outer',axis=1)
print len(df)
for row in df.loc[df.male.isnull(), 'male'].index:
    df.at[row, 'male'] = []
for row in df.loc[df.female.isnull(), 'female'].index:
    df.at[row, 'female'] = [] 
    
def check_in_oposit_gender(diff_p,prof,ab,gender):
    wrong_gender_dict={}
    if gender=="male":
        oposit_gender="female"
    elif gender=="female":
        oposit_gender="male"
    links=ab[prof][oposit_gender+"_person"]
    new_links=map(lambda x: re.sub(r'\([^)]*\)', '', x.replace("_"," ").replace("&nbsp;"," ").replace(".","")).strip(), links)
    wrong_gender=[]
    for i in diff_p:
        if i in new_links:
            #print i,",gender=",gender,", but should be",oposit_gender,prof
            wrong_gender.append(i)
        else:
            #Lauren!=Laurent,Jona!=Jonathan but Jona is male not female,
            if ((i=="Lauren")&(gender=="female")&(prof=="Model")):
                pass
            elif ((i=="Jona")&(gender=="female")&(prof=="Missionar")):
                wrong_gender.append(i)
                #print i,",gender=",gender,", but should be",oposit_gender,prof
            else:
                wrong_gender_dict.update({i:j for j in new_links if ((i in j)|(j in i))})
    #if len(wrong_gender_dict)>0:
        #print wrong_gender_dict,prof 
    return wrong_gender,wrong_gender_dict
    
def ident_overlap(df,gender):
    person_m=defaultdict(list)
    ab=load_simple_json('de/wiki/all_data_from pages.json')   
    abos=pd.DataFrame.from_dict(ab,orient="index")
    both_empty=0
    links_empty=0
    polyglot_empty=0
    all_incl=0
    gesamt_all=[]
    wrong_gender=defaultdict(list)
    for prof in ab:
        links_m=ab[prof][gender+"_person"]
        polyglot_m=df.loc[[prof],gender].values[0]
        diff_p=[]
        diff_l=[]
        #print len(links_m)
        #print len(polyglot_m)
        if (len(polyglot_m)>0) & (len(links_m)>0):
            #check all elements 
            new_polyglot_m=map(lambda x: re.sub(r'\([^)]*\)', '', x.replace("_"," ").replace("&nbsp;"," ").replace(".","").replace(" - ","-")).strip(), polyglot_m) 
            new_links_m=map(lambda x: re.sub(r'\([^)]*\)', '', x.replace("_"," ").replace("&nbsp;"," ").replace(".","")).strip(), links_m)        
            gesamt=[i for i in new_links_m if (i in new_polyglot_m)]
            percent=100*float(len(gesamt))/len(new_links_m)
            #print percent,"% ===",gesamt
            if percent==100:
                gesamt_all.extend(gesamt)
                all_incl+=1
                person_m[prof]=new_polyglot_m
                diff_p=set(new_polyglot_m)-set(gesamt)
                #print "100%="
            else:
                #save difference   
                diff_p=set(new_polyglot_m)-set(gesamt)
                diff_l=set(new_links_m)-set(gesamt)
                for i in diff_l:
                    a={i:j for j in diff_p if ((i in j)|(j in i))}
                    diff_l=diff_l-set(a.keys())
                    diff_p=diff_p-set(a.values())
                    gesamt.extend(a.keys())
                percent=100*float(len(gesamt))/len(new_links_m)
                if percent==100:
                    gesamt_all.extend(gesamt)
                    all_incl+=1
                    person_m[prof]=gesamt+list(diff_p)
                    #print "100%="
                elif len(diff_p)==0:
                    gesamt_all.extend(gesamt)
                    person_m[prof]=gesamt+list(diff_l)
                    #print "0_pol=="
                else:
                    for i in diff_l:
                        if len(i.split())>1:
                            i_1=i.split()[0]
                            i_n=i.split()[-1:]
                            for j in diff_p:
                                if len(j.split())>1:
                                    if ((j.split()[0]==i_1)&(j.split()[-1:]==i_n))|((j.split()[0]==i_n[0])&(j.split()[-1:][0]==i_1)):
                                        diff_l=diff_l-set([i])
                                        diff_p=diff_p-set([j])
                                        gesamt.append(i)
                    percent=100*float(len(gesamt))/len(new_links_m)
                    if percent==100:
                        gesamt_all.extend(gesamt)
                        all_incl+=1
                        person_m[prof]=gesamt+list(diff_p)
                        #print "100%="
                    elif len(diff_p)==0:
                        gesamt_all.extend(gesamt)
                        person_m[prof]=gesamt+list(diff_l)
                        #print "0_pol=="
                    else:
                        #print diff_p
                        #print diff_l
                        #print percent,"& ================================"
                        gesamt_all.extend(gesamt)
                        person_m[prof]=gesamt + list(diff_l)+ list(diff_p)
                        #print "other=="
                        #TODO:levenstein distance
                        #TODO: check the text
            #print len(gesamt)
            #print len(person_m[prof])
            #check diff_p in oposite gender list
            if len(diff_p)>0:
                wr,wr_d=check_in_oposit_gender(diff_p,prof,ab,gender)
                if (len(wr)>0)|(len(wr_d)>0):
                    wrong_gender[prof]=[wr,wr_d]
                    #delete from person_m[prof] all wr values, add to gesamt_opposit all wr values
                    #delete from person_m[prof] wr_d.keys(), add to gesamt_oposit wr_d.values()
        elif len(polyglot_m)>0:
            links_empty+=1
            person_m[prof]=polyglot_m
            #print "only_p=="
            #print "0"
            #print len(person_m[prof])
            #check polyglot_m in female_links
            wr,wr_d=check_in_oposit_gender(polyglot_m,prof,ab,gender)
            if (len(wr)>0)|(len(wr_d)>0):
                wrong_gender[prof]=[wr,wr_d]
        elif (len(links_m)>0):
            polyglot_empty+=1
            person_m[prof]=map(lambda x: x.replace("_"," "), links_m)
            #print "only_l=="
            #print "0"
            #print len(person_m[prof])
        else:
            both_empty+=1

    sum_empty=both_empty+polyglot_empty+links_empty
    non_empty=len(df)-sum_empty
    print "100% includes in ",all_incl,"over", non_empty
    print "Both empty:",both_empty,"Polyglot empty:",polyglot_empty,"Links empty:",links_empty,";Sum=",sum_empty,"(",non_empty,")"
    return person_m,gesamt_all,ab,wrong_gender



person_m,gesamt_all_m,ab_m,wrong_gender_m=ident_overlap(df,"male")
print len(person_m)
print wrong_gender_m
print"======================"
person_f,gesamt_all_f,ab_f,wrong_gender_f=ident_overlap(df,"female")
print len(person_f)
print wrong_gender_f

394 394
885
100% includes in  93 over 227
Both empty: 492 Polyglot empty: 17 Links empty: 149 ;Sum= 658 ( 227 )
393
defaultdict(<type 'list'>, {u'Bibliothekar': [[], {u'Costa Greene': u'Belle da Costa Greene'}], u'Professor': [[u'Gabriele Possanner'], {}], u'Online-Redakteur': [[u'Gabriele Hooffacker'], {}], u'Souffleur': [[u'Gabriele Auenm\xfcller'], {}], u'Journalist': [[u'Gabriele Hooffacker'], {u'Gabriele Krone': u'Gabriele Krone-Schmalz'}], u'Model': [[u'Shalom Harlow', u'Dorian Leigh', u'Jerry Hall'], {u'Abdulmajid': u'Iman Abdulmajid'}], u'Redakteur': [[u'Gabriele Hooffacker'], {}], u'Weber': [[], {u'Alen M\xfcller': u'Alen M\xfcller-Hellwig'}]})
100% includes in  58 over 90
Both empty: 670 Polyglot empty: 12 Links empty: 113 ;Sum= 795 ( 90 )
215
defaultdict(<type 'list'>, {u'Bauchredner': [[u'Sascha Grammel'], {}], u'Stuckateur': [[u'Andrea Gallasini'], {}], u'Zusteller': [[u'Gaby Minneboo'], {}], u'Dirigent': [[u'Jean-Baptiste Lully'], {}], u'Stuntman': [[u'Miki Dora', u'Dar R

In [3]:
#delete from person_m[prof] all wrong_gender[prof][0] values, add to gesamt_opposit all wrong_gender[prof][0] values
#delete from person_m[prof] wrong_gender[prof][1].keys(), add to gesamt_oposit wrong_gender[prof][1].values()
was_male_become_female=0
for prof in wrong_gender_m:
    for i in wrong_gender_m[prof][0]:
        person_m[prof].remove(i)
        gesamt_all_f.append(i)
        was_male_become_female+=1
    for key in wrong_gender_m[prof][1]:
        person_m[prof].remove(key)
        was_male_become_female+=1
    gesamt_all_f.extend(wrong_gender_m[prof][1].values())
print "False Positives in male:",was_male_become_female

#delete from person_f[prof] all wrong_gender[prof][0] values, add to gesamt_opposit all wrong_gender[prof][0] values
#delete from person_f[prof] wrong_gender[prof][1].keys(), add to gesamt_oposit wrong_gender[prof][1].values()
was_female_become_male=0
for prof in wrong_gender_f:
    for i in wrong_gender_f[prof][0]:
        person_f[prof].remove(i)
        gesamt_all_m.append(i)
        was_female_become_male+=1
    for key in wrong_gender_f[prof][1]:
        person_f[prof].remove(key)
        was_female_become_male+=1
    gesamt_all_m.extend(wrong_gender_f[prof][1].values())
#,len(wrong_gender_m), wrong_gender_f,len(wrong_gender_f)
print "False Positives in female:",was_female_become_male

False Positives in male: 12
False Positives in female: 51


In [8]:
df.female_amount.sum()

774.0

In [4]:
##male
#print "Shared by both lists: 2*",len(gesamt_all)+19
sh=2*(len(gesamt_all_m)+19)
#n_l=sum([ab[i]["male_person_amount"] for i in ab]) #list ab DATASET1
n_l=df.male_person_amount.sum()
print n_l
n_p=df.male_amount.sum() -was_male_become_female#df.male #DATASET2
print n_p
n_l_p=n_l+n_p
pr_sh=round(100*float(sh)/n_l_p)
pr_n_l=round(100*float(sh/2)/n_l)
pr_n_p=round(100*float(sh/2)/n_p)
print n_l_p-sh/2
print "We had overall: ",n_l_p,"male persons in 2 DATASETS."
print "Among them",sh,"(",pr_sh,"% ) are shared.It is",pr_n_l,"% of DATASET1(links) and ",pr_n_p,"% of DATASET2(polyglot)"

#l=sum([len(person_m[i]) for i in person_m])#4303
#print l#4303
#+l+sh/2-19#6235

2685
3536.0
4219.0
We had overall:  6221.0 male persons in 2 DATASETS.
Among them 4004 ( 64.0 % ) are shared.It is 75.0 % of DATASET1(links) and  57.0 % of DATASET2(polyglot)


In [5]:
#female
sh_f=2*(len(gesamt_all_f)+7)
n_l_f=df.female_person_amount.sum()#list ab DATASET1
print n_l_f
n_p_f=df.female_amount.sum() -was_female_become_male#df.male #DATASET2
print n_p_f

n_l_p_f=n_l_f+n_p_f
pr_sh_f=round(100*float(sh_f)/n_l_p_f)
pr_n_l_f=round(100*float(sh_f/2)/n_l_f)
pr_n_p_f=round(100*float(sh_f/2)/n_p_f)
print n_l_p_f-sh_f/2
print "We had overall: ",n_l_p_f,"female persons in 2 DATASETS."
print "Among them",sh_f,"(",pr_sh_f,"% ) are shared.It is",pr_n_l_f,"% of DATASET1(links) and ",pr_n_p_f,"% of DATASET2(polyglot)"


347
723.0
801.0
We had overall:  1070.0 female persons in 2 DATASETS.
Among them 538 ( 50.0 % ) are shared.It is 78.0 % of DATASET1(links) and  37.0 % of DATASET2(polyglot)


In [120]:
pr=round(100*float(sh_f+sh)/(n_l_p_f+n_l_p))
pr_l=round(100*float((sh_f+sh)/2)/(n_l_f+n_l))
pr_p=round(100*float((sh_f+sh)/2)/(n_p_f+n_p))
print "We have overall: ",n_l_p_f+n_l_p,"persons in 2 DATASETS."
print "Among them",sh_f+sh,"(",pr,"% ) are shared.It is",pr_l,"% of DATASET1(links) and ",pr_p,"% of DATASET2(polyglot)"


We have overall:  7291.0 persons in 2 DATASETS.
Among them 4542 ( 62.0 % ) are shared.It is 75.0 % of DATASET1(links) and  53.0 % of DATASET2(polyglot)


In [123]:
print sum(len(person_m[prof])for prof in person_m)-19
print sum(len(person_f[prof])for prof in person_f)-7
together=sum(len(person_m[prof])for prof in person_m)-19+sum(len(person_f[prof])for prof in person_f)-7
together_share=(sh_f+sh)/2
print "Now we have",together," where ",together_share," are shared"

4272
813
Now we have 5085  where  2271  are shared


The overlap of 2 Datasets is 62% (2271 unique persons), which is 75% of Dataset1(the links) and 53% of DATASET2(polyglot).

In [130]:
df["male_all"]=df.apply(lambda x: person_m[x.name], axis = 1)
df["female_all"]=df.apply(lambda x: person_f[x.name], axis = 1)
df["male_all_amount"]=df.apply(lambda x: len(person_m[x.name]), axis = 1)
df["female_all_amount"]=df.apply(lambda x: len(person_f[x.name]), axis = 1)

In [132]:
df.to_csv('de/all_data_bias_labour_gender_all_persons.csv', sep='\t', encoding='utf-8')

In [133]:
df

,male_person_amount,all_link,lead,text,male_person,female_person,female_person_amount,Bias,prof_gender,labor_market_domin,profession,male,female_amount,female,male_amount,male_all,female_all,male_all_amount,female_all_amount
Abfallbeauftragter,0,"[Unternehmer, Kreislaufwirtschaft, Abfallbesei...",Ein Abfallbeauftragter oder Betriebsbeauftragt...,Ein Abfallbeauftragter oder Betriebsbeauftragt...,[],[],0,male,male,male,Abfallbeauftragter,"[Schmidt, Stephan, Erich]",NaN,[],3,"[Schmidt, Stephan, Erich]",[],3,0
Abt,4,"[Kirchliche_Heraldik, Galero, Chorkleidung, Ha...","Ein Abt (von spätlat. abbas, aus aram. abba „V...","Ein Abt (von spätlat. abbas, aus aram. abba „V...","[Karl_Martell, Leo_X., Franz_I._(Frankreich), ...",[],0,male,male,female,Abt,"[Leo XIII, Papst Leo X, Schmitt, Karl Martell,...",1,[Martina Wiech],5,"[Karl Martell, Leo XIII, Leo X, Franz I, Schmitt]",[Martina Wiech],5,1
Account-Manager,0,"[Vertrieb, Kunde, Unternehmen, Key-Account-Man...","Der Account-Manager (AM, deutsch: Kundenbetreu...","Der Account-Manager (AM, deutsch: Kundenbetreu...",[],[],0,male,male,male,Account-Manager,[],NaN,[],NaN,[],[],0,0
Altenpflegehelfer,0,"[Hessen, Nordrhein-Westfalen, Baden-Württember...",Altenpflegehilfe mit der Berufsbezeichnung Alt...,Altenpflegehilfe mit der Berufsbezeichnung Alt...,[],[],0,male,male,female,Altenpflegehelfer,"[Thomas Klie, Robert, Bernd Hein]",1,[Virginia Guerra],3,"[Thomas Klie, Robert, Bernd Hein]",[Virginia Guerra],3,1
Altenpfleger,0,"[Ambulante_Pflege, Häusliche_Krankenpflege, So...",Altenpfleger und Altenpflegerinnen pflegen und...,Altenpfleger und Altenpflegerinnen pflegen und...,[],[],0,male,male,female,Altenpfleger,"[Manfred Engl, Frank Drieschner]",3,"[Brigitte Zellhuber, Annette Riedel, Sonja Sch...",2,"[Manfred Engl, Frank Drieschner]","[Brigitte Zellhuber, Annette Riedel, Sonja Sch...",2,3
Amtsanwalt,0,"[Siegelmarke, Beamter_(Deutschland), Gehobener...",Ein Amtsanwalt ist in Deutschland ein Beamter ...,Ein Amtsanwalt ist in Deutschland ein Beamter ...,[],[],0,male,male,female,Amtsanwalt,[],NaN,[],NaN,[],[],0,0
Angestellter,7,"[Arbeitnehmer, Arbeiter, Arbeitsrecht, Sozialr...","Ein Angestellter, auch Privatangestellter, ist...","Ein Angestellter, auch Privatangestellter, ist...","[Fritz_Croner, Michel_Crozier, Arnd_Zschiesche...",[Irmgard_Keun],1,male,male,NaN,Angestellter,"[Ulf Kadritzke, Fritz Croner, Ruprecht, Charle...",3,"[Lang, Irmgard, Tatjana Timoschenko]",14,"[Ulf Kadritzke, Fritz Croner, Ruprecht, Charle...","[Irmgard Keun, Lang, Tatjana Timoschenko]",14,3
Animateur,0,"[Unterhaltungskünstler, Tourismus, Tagung, Ver...","Animateur (weibliche Form: Animateurin, von fr...","Animateur (weibliche Form: Animateurin, von fr...",[],[],0,male,male,female,Animateur,[],NaN,[],NaN,[],[],0,0
Anlagenelektriker,0,"[Berufsausbildung, Österreich, Elektrotechnik_...",Anlagenelektriker war ein Lehrberuf in Österre...,Anlagenelektriker war ein Lehrberuf in Österre...,[],[],0,male,male,male,Anlagenelektriker,[],NaN,[],NaN,[],[],0,0
Anlagenmechaniker,0,"[Ausbildungsberuf, Berufsbildungsgesetz_(Deuts...",Der Anlagenmechaniker ist in Deutschland ein s...,Der Anlagenmechaniker ist in Deutschland ein s...,[],[],0,male,male,male,Anlagenmechaniker,[],NaN,[],NaN,[],[],0,0


In [ ]:
import pickle
with open('de/all_not_person_polyglot.pckl') as f:
    all_other,all_other_1,all_other_2,all_other_none = pickle.load(f)
print all_other, len(all_other)#18
print all_other_2,len(all_other_2)#77
print all_other_none,len(all_other_none)#68